In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_14929: Some(http://gw02.itversity.com:4041)

In [2]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1533622723243_14929


In [3]:
// in Scala
import org.apache.spark.sql.expressions.MutableAggregationBuffer
import org.apache.spark.sql.expressions.UserDefinedAggregateFunction
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

class BoolAnd extends org.apache.spark.sql.expressions.UserDefinedAggregateFunction {
    
  def inputSchema: StructType =
    StructType(StructField("value", BooleanType) :: Nil)
  def bufferSchema: StructType = StructType(
    StructField("result", BooleanType) :: Nil
  )
  def dataType: DataType = BooleanType
  def deterministic: Boolean = true
  def initialize(buffer: MutableAggregationBuffer): Unit = {
    buffer(0) = true
  }
  def update(buffer: MutableAggregationBuffer, input: Row): Unit = {
    buffer(0) = buffer.getAs[Boolean](0) && input.getAs[Boolean](0)
  }
  def merge(buffer1: MutableAggregationBuffer, buffer2: Row): Unit = {
    buffer1(0) = buffer1.getAs[Boolean](0) && buffer2.getAs[Boolean](0)
  }
  def evaluate(buffer: Row): Any = {
    buffer(0)
  }
}

defined class BoolAnd


In [30]:
// COMMAND ----------
val ba = new BoolAnd()
spark.udf.register("booland", ba)

ba = BoolAnd@4a276b8c


$line19.$read$$iw$$iw$BoolAnd@4a276b8c

In [31]:
val df1 = spark.range(1)

df1 = [id: bigint]


[id: bigint]

In [32]:
df1.show()

+---+
| id|
+---+
|  0|
+---+



In [33]:
val df2 = df1.selectExpr("id","explode(array(TRUE, TRUE, TRUE)) as t")

df2 = [id: bigint, t: boolean]


[id: bigint, t: boolean]

In [34]:
df2.show()

+---+----+
| id|   t|
+---+----+
|  0|true|
|  0|true|
|  0|true|
+---+----+



In [35]:
val df3 = df2.selectExpr("id","t","explode(array(TRUE, FALSE, TRUE)) as f")

df3 = [id: bigint, t: boolean ... 1 more field]


[id: bigint, t: boolean ... 1 more field]

In [36]:
df3.show()

+---+----+-----+
| id|   t|    f|
+---+----+-----+
|  0|true| true|
|  0|true|false|
|  0|true| true|
|  0|true| true|
|  0|true|false|
|  0|true| true|
|  0|true| true|
|  0|true|false|
|  0|true| true|
+---+----+-----+



In [43]:
val df4 =  df3.select(ba(col("t")), expr("booland(f)"))

df4 = [booland(t): boolean, booland(f): boolean]


[booland(t): boolean, booland(f): boolean]

In [44]:
df4.show()
// COMMAND ----------

+----------+----------+
|booland(t)|booland(f)|
+----------+----------+
|      true|     false|
+----------+----------+

